in this project I am going to read an excel file and write the data in to mysql or any other DBMS using python .
after that I create an elasticsearch and upload the MySql data to elasticsearch.

In [ ]:
#pip install mysql-connector #to install mysql-connnectors   

In [23]:
#importing libraries
import mysql.connector as connection  
import xlrd
import pandas as pd

In [14]:
#reading the excel files which have to sheet in a single excel book
book = xlrd.open_workbook("beginner_assignment.xlsx")
sheet = book.sheet_by_name("group_listing")
sheet1 = book.sheet_by_name("product_listing")

In [15]:
conn = connection.connect(host = "localhost", user = "root",password="")  #connection string to mysql database
cursor = conn.cursor(buffered=True,dictionary=True)
#------------- here we are creating the database(assignment) in and the tables in side our database
try:  
    cursor.execute('CREATE DATABASE IF NOT EXISTS assignment')
    cursor.execute('USE assignment')
    cursor.execute("create table IF NOT EXISTS group_listing(groupName varchar(30) primary key, groupDescription varchar(30) not null , isActive varchar(5) not null)")  
    cursor.execute("create table IF NOT EXISTS product_listing(productName varchar(30) not null, modelName varchar(30) not null , productSerialNo varchar(30) not null, groupAssociated varchar(30) not null,productMRP varchar(30) not null   , FOREIGN KEY(groupAssociated) REFERENCES group_listing(groupName))")  
except:
    conn.rollback()
# conn.close()  

In [10]:
#---------------- write the sql query to insert values which is read from excel file
query = """INSERT INTO group_listing (groupName, groupDescription, isActive) VALUES (%s, %s, %s)"""
query1 = """INSERT INTO product_listing (productName, modelName, productSerialNo, groupAssociated, productMRP) VALUES (%s, %s, %s, %s, %s)"""
for r in range(1, sheet.nrows):
    groupName= sheet.cell(r,0).value
    groupDescription= sheet.cell(r,1).value
    isActive= sheet.cell(r,2).value
    values = (groupName, groupDescription, isActive)
    # Execute sql Query
    cursor.execute(query, values)
    
for r1 in range(1, sheet1.nrows):
    productName= sheet1.cell(r,0).value
    modelName= sheet1.cell(r,1).value
    productSerialNo= sheet1.cell(r,2).value
    groupAssociated= sheet1.cell(r,3).value
    productMRP = sheet1.cell(r,4).value
    values1 = (productName,modelName, productSerialNo,groupAssociated,productMRP)
    # Execute sql Query
    cursor.execute(query1, values1)
    
# Close the cursor
cursor.close()

# Commit the transaction
conn.commit()

# Close the database connection


In [20]:
# task2  creating the Elasticsearch

In [22]:
#first of all we have to install elastic search library
#pip install elasticsearch

In [24]:
from elasticsearch import 

In [34]:
# here I create the elasticsearch in this server . but you can create on local server also as you have to give the host name and port number
es = Elasticsearch('http://101.53.136.181:9200')

In [28]:
es.indices.create(index='assignment' , ignore=400) # creating the index in elasticsearch

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [assignment/Jbl0VB44SfKCSKq9ES1gMA] already exists',
    'index_uuid': 'Jbl0VB44SfKCSKq9ES1gMA',
    'index': 'assignment'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [assignment/Jbl0VB44SfKCSKq9ES1gMA] already exists',
  'index_uuid': 'Jbl0VB44SfKCSKq9ES1gMA',
  'index': 'assignment'},
 'status': 400}

In [30]:
es.indices.exists(index='assignment') #checking for exisitance of index in elasticsearch if the index exist the out put will be true

True

In [31]:
#selecting data from Mysql and inserting them to elasticsearch using the index which we created above 
try:
    cursor.execute('select * from group_listing')
    result = cursor.fetchall()
    counter=1
    for x in result:
        es.index(index='assignment' , doc_type='places' , id=counter , body=x)
        counter=counter+1
except:
    conn.rollback()
    

In [33]:
# searching the value from elasticsearch as we have already inserted 
res = es.get(index='assignment' , doc_type="places" , id=2)
res

{'_index': 'assignment',
 '_type': 'places',
 '_id': '2',
 '_version': 1,
 'found': True,
 '_source': {'groupName': 'Group 2',
  'groupDescription': 'Group Number 2',
  'isActive': 'yes'}}